In [ ]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.datasets import load_boston

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

In [ ]:
## LOAD DADASET
boston_dataset = load_boston()
boston_data = boston_dataset['data']                  # NDARRAY
boston_feature_names = boston_dataset['feature_names']
boston_df = pd.DataFrame(data=boston_data, columns=boston_feature_names)
boston_df['price']= boston_dataset['target']


train_x, test_x, train_y, test_y = train_test_split(boston_df[boston_dataset['feature_names']], boston_df['price'], random_state=0, test_size=0.33)



train_df = train_x.copy()
train_df['price']=train_y


test_df = test_x.copy()
test_df['price']=test_y

In [ ]:
## H2O FRAME으로 변환
train_hf = h2o.H2OFrame(train_df)
test_hf = h2o.H2OFrame(test_df)
target_col = 'price'
feature_col = hf.columns.remove(target_col)


In [ ]:
## H2O AUTOML 객체 생성

aml = H2OAutoML(max_models=10,
               nfolds=4,
               exclude_algos=['DeepLearning'],
               keep_cross_validation_predictions=True,
               seed=5)

In [ ]:
## 학습진행
aml.train(x=feature_col,
         y=target_col,
         training_frame=train_hf,
         leaderboard_frame=test_hf)

In [ ]:
print(aml.leaderboard)

In [ ]:
# 동일기준으로 검증

from sklearn.metrics import mean_squared_error

h_pred = aml.predict(test_hf).as_data_frame()

mean_squared_error(test_y, h_pred, squared=False)

In [ ]:
aml.leader